In [1]:
import numpy as np; import matplotlib.pyplot as plt;
import ipywidgets as widgets; from ipywidgets import interact;
from ipywidgets.widgets import *
import numpy.random as rnd; from numpy import *;
from shared import *;
from matplotlib import rc;
rc('text', usetex=True)

In [82]:
def vizualizacija_gauss(mean, var, N=100):
    x = linspace(-8, 8, N)
    p = gaussian(x, mean, var)
    val_at_var = gaussian(mean + var, mean, var) # Vrijednost raspodjele u tački x=var
    
    # Crtanje raspodjele
    plt.plot(x, p, label='$p_X(x)$')
    # Vizualizacija očekivane vrijednosti
    plt.plot([mean, mean], list(plt.ylim()), label='Očekivana vrijednost')
    # Vizualizacija varijanse
    plt.annotate(xy=(mean, val_at_var), xytext=(mean+var, val_at_var),
                 text='', arrowprops=dict(arrowstyle='<->', shrinkA=0, shrinkB=0))
    plt.plot([mean, mean+var], [val_at_var, val_at_var],
             linewidth=0.8, c='k', label='Varijansa')
    
    plt.xlabel('$x$'); plt.legend(); plt.show()
    
interact(vizualizacija_gauss,
         mean=FloatSlider(0, min=-5, max=5),
         var=FloatSlider(1, min=0.8, max=5),
         N=IntSlider(70, min=25, max=10000));

interactive(children=(FloatSlider(value=0.0, description='mean', max=5.0, min=-5.0), FloatSlider(value=1.0, de…

In [79]:
def plot_multivariate_gaussian(mean, cov, N=100):
    # Generisanje vektora ulaznih varijabli
    
    elipsa = cov_elipsa(mean, cov)
    # Određivanje prihvatljivih granica oblasti
    xlim = [mean[0]-elipsa[2]*1.8, mean[0]+elipsa[2]*1.8]
    ylim = [mean[1]-elipsa[3]*1.8, mean[1]+elipsa[3]*1.8]
    
    # Generisanje vektora koordinata
    x = linspace(xlim[0], xlim[1], N)
    y = linspace(ylim[0], ylim[1], N)
    _X,_Y = meshgrid(x, y)
    X = _X.flatten(); Y = _Y.flatten()

    # Računanje raspodjele
    p = multivariate_gaussian(stack((X,Y)), mean, cov)
    
    #################

    fig = plt.figure(figsize=(4,8.7))
    ax = fig.add_subplot(2,1,1, projection='3d')
    
    # Crtanje raspodjele
    ax.plot_surface(_X, _Y, p.reshape(N,N), cmap='hot')
    
    # Crtanje marginalnih raspodjela
    ax.plot3D(X, ylim[1]*ones_like(X),
              gaussian(X, mean[0], cov[0][0]) * sqrt(cov[0][0]/(2*pi*linalg.det(cov))),
              c='orchid', label='$p_X(x)$')
    ax.plot3D(xlim[0]*ones_like(Y), Y,
              gaussian(Y, mean[1], cov[1][1]) * sqrt(cov[1][1]/(2*pi*linalg.det(cov))),
              c='dodgerblue', label='$p_Y(y)$')
    
    plt.xlabel('$x$'); plt.ylabel('$y$')
    plt.legend(bbox_to_anchor=(0.7,1))
    
    # Crtanje konturnog grafika
    ax = fig.add_subplot(2,1,2)
    ax.contourf(_X, _Y, p.reshape(N,N), 50, cmap='hot', antialiased=False)
    
    # Srednja vrijednost
    ax.scatter(mean[0], mean[1], c='k', label='Očekivanje')
    
    # Elipsa kovarijanse
    ax.plot(elipsa[0], elipsa[1], label='Elipsa kovarijanse')
    
    plt.legend(bbox_to_anchor=(1.02,1.02), framealpha=1); plt.show()
    step = diff(xlim)*diff(ylim)/N**2

In [80]:
def vizualizacija_multivar_gauss(meanX, meanY, varX, varY, covXY, N=100):
    mean = [meanX, meanY]
    cov = [[varX, covXY],[covXY, varY]]
    plot_multivariate_gaussian(mean, cov, N)
    
meanX=FloatSlider(0, min=-10, max=10, step=0.05)
meanY=FloatSlider(0, min=-10, max=10, step=0.05)
varX=FloatSlider(1, min=0.001, max=20, step=0.05)
varY=FloatSlider(1, min=0.001, max=20, step=0.05)
covXY=FloatSlider(0, min=-0.9999, max=0.9999, step=0.05)
N=IntSlider(30, min=1, max=200)

def on_change(change):
    val = sqrt(varX.value * varY.value)
    covXY.min = -val + 0.001
    covXY.max = val - 0.001
    
varX.observe(on_change, 'value')
varY.observe(on_change, 'value')
    
interact(vizualizacija_multivar_gauss, meanX=meanX, meanY=meanY,
         varX=varX, varY=varY, covXY=covXY, N=N);

interactive(children=(FloatSlider(value=0.0, description='meanX', max=10.0, min=-10.0, step=0.05), FloatSlider…